In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [3]:
bank = pd.read_csv("C:\Aalesh and Mandar\Datasets\Cases\Bankruptcy\Bankruptcy.csv")

In [4]:
bank.head()

,Class,region.centroid.col,region.centroid.row,region.pixel.count,short.line.density.5,short.line.density.2,vedge.mean,vegde.sd,hedge.mean,hedge.sd,intensity.mean,rawred.mean,rawblue.mean,rawgreen.mean,exred.mean,exblue.mean,exgreen.mean,value.mean,saturation.mean,hue-mean
0,BRICKFACE,188,133,9,0.0,0.0,0.333333,0.266667,0.500000,0.077778,6.666666,8.333334,7.777778,3.888889,5.000000,3.333333,-8.333333,8.444445,0.538580,-0.924817
1,BRICKFACE,105,139,9,0.0,0.0,0.277778,0.107407,0.833333,0.522222,6.111111,7.555555,7.222222,3.555556,4.333334,3.333333,-7.666666,7.555555,0.532628,-0.965946
2,BRICKFACE,34,137,9,0.0,0.0,0.500000,0.166667,1.111111,0.474074,5.851852,7.777778,6.444445,3.333333,5.777778,1.777778,-7.555555,7.777778,0.573633,-0.744272
3,BRICKFACE,39,111,9,0.0,0.0,0.722222,0.374074,0.888889,0.429629,6.037037,7.000000,7.666666,3.444444,2.888889,4.888889,-7.777778,7.888889,0.562919,-1.175773
4,BRICKFACE,16,128,9,0.0,0.0,0.500000,0.077778,0.666667,0.311111,5.555555,6.888889,6.666666,3.111111,4.000000,3.333333,-7.333334,7.111111,0.561508,-0.985811


In [29]:
bank = bank.drop("NO", axis=1)
bank = bank.drop("YR", axis=1)

In [30]:
x = bank.drop("D", axis=1)
y = bank["D"]

In [31]:
x_train, x_test, y_train, y_test = train_test_split(x, y, stratify=y, random_state = 2022, test_size = 0.3)

In [32]:
lr = LogisticRegression()
lr.fit(x_train, y_train)

C:\Users\admin\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [33]:
y_pred = lr.predict(x_test)
y_pred_prob = lr.predict_proba(x_test)[:,1]

In [34]:
y_pred_prob

array([0.15713683, 0.98755942, 0.21791388, 0.71543636, 0.91368156,
       0.10966542, 0.76312731, 0.2285903 , 0.99988327, 0.73190131,
       0.98694843, 0.2776136 , 0.12810502, 0.74279542, 1.        ,
       0.00831932, 0.22224873, 0.06374418, 0.92214184, 0.42711131,
       0.20511652, 0.99613898, 0.98560248, 0.35968104, 0.37319428,
       0.27283447, 0.97351151, 0.31999922, 0.21283222, 0.40923389,
       0.52056465, 0.00120111, 0.90380277, 0.04724935, 0.22605232,
       0.93276138, 0.93076694, 0.21534177, 0.25950783, 0.18973778])

In [35]:
print(accuracy_score(y_test, y_pred))
print(roc_auc_score(y_test, y_pred_prob))

0.825
0.8025000000000001


In [36]:
scaler = StandardScaler()

In [37]:
kfold = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 2022)
lr = LogisticRegression(solver = "saga", random_state = 100)

pipe = Pipeline([("STD", scaler),("LR",lr)])
print(pipe.get_params())

params = {"LR__penalty": ["l1", "l2", "elasticnet", None],
          "LR__C" : np.linspace(0.001, 4, 5),
          "LR__l1_ratio" : np.linspace(0.001, 1, 5)}

{'memory': None, 'steps': [('STD', StandardScaler()), ('LR', LogisticRegression(random_state=100, solver='saga'))], 'verbose': False, 'STD': StandardScaler(), 'LR': LogisticRegression(random_state=100, solver='saga'), 'STD__copy': True, 'STD__with_mean': True, 'STD__with_std': True, 'LR__C': 1.0, 'LR__class_weight': None, 'LR__dual': False, 'LR__fit_intercept': True, 'LR__intercept_scaling': 1, 'LR__l1_ratio': None, 'LR__max_iter': 100, 'LR__multi_class': 'auto', 'LR__n_jobs': None, 'LR__penalty': 'l2', 'LR__random_state': 100, 'LR__solver': 'saga', 'LR__tol': 0.0001, 'LR__verbose': 0, 'LR__warm_start': False}


In [40]:
gcv = GridSearchCV(pipe, param_grid = params, verbose = 3, scoring = "roc_auc", cv = kfold)
gcv.fit(x, y)

Fitting 5 folds for each of 100 candidates, totalling 500 fits
[CV 1/5] END LR__C=0.001, LR__l1_ratio=0.001, LR__penalty=l1;, score=0.500 total time=   0.0s
[CV 2/5] END LR__C=0.001, LR__l1_ratio=0.001, LR__penalty=l1;, score=0.500 total time=   0.0s
[CV 3/5] END LR__C=0.001, LR__l1_ratio=0.001, LR__penalty=l1;, score=0.500 total time=   0.0s
[CV 4/5] END LR__C=0.001, LR__l1_ratio=0.001, LR__penalty=l1;, score=0.500 total time=   0.0s
[CV 5/5] END LR__C=0.001, LR__l1_ratio=0.001, LR__penalty=l1;, score=0.500 total time=   0.0s
[CV 1/5] END LR__C=0.001, LR__l1_ratio=0.001, LR__penalty=l2;, score=0.813 total time=   0.0s
[CV 2/5] END LR__C=0.001, LR__l1_ratio=0.001, LR__penalty=l2;, score=0.780 total time=   0.0s
[CV 3/5] END LR__C=0.001, LR__l1_ratio=0.001, LR__penalty=l2;, score=0.858 total time=   0.0s
[CV 4/5] END LR__C=0.001, LR__l1_ratio=0.001, LR__penalty=l2;, score=0.953 total time=   0.0s
[CV 5/5] END LR__C=0.001, LR__l1_ratio=0.001, LR__penalty=l2;, score=0.982 total time=   0.

C:\Users\admin\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1476: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
C:\Users\admin\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1476: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
C:\Users\admin\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1476: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
C:\Users\admin\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1476: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
C:\Users\admin\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1476: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
C:\Users\admin\anaconda3\lib\site-packag

[CV 2/5] END LR__C=0.001, LR__l1_ratio=0.001, LR__penalty=elasticnet;, score=0.797 total time=   0.0s
[CV 3/5] END LR__C=0.001, LR__l1_ratio=0.001, LR__penalty=elasticnet;, score=0.864 total time=   0.0s
[CV 4/5] END LR__C=0.001, LR__l1_ratio=0.001, LR__penalty=elasticnet;, score=0.959 total time=   0.0s
[CV 5/5] END LR__C=0.001, LR__l1_ratio=0.001, LR__penalty=elasticnet;, score=0.982 total time=   0.0s
[CV 1/5] END LR__C=0.001, LR__l1_ratio=0.001, LR__penalty=None;, score=nan total time=   0.0s
[CV 2/5] END LR__C=0.001, LR__l1_ratio=0.001, LR__penalty=None;, score=nan total time=   0.0s
[CV 3/5] END LR__C=0.001, LR__l1_ratio=0.001, LR__penalty=None;, score=nan total time=   0.0s
[CV 4/5] END LR__C=0.001, LR__l1_ratio=0.001, LR__penalty=None;, score=nan total time=   0.0s
[CV 5/5] END LR__C=0.001, LR__l1_ratio=0.001, LR__penalty=None;, score=nan total time=   0.0s
[CV 1/5] END LR__C=0.001, LR__l1_ratio=0.25075, LR__penalty=l1;, score=0.500 total time=   0.0s
[CV 2/5] END LR__C=0.001, 

C:\Users\admin\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1476: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
C:\Users\admin\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1476: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
C:\Users\admin\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1476: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
C:\Users\admin\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1476: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
C:\Users\admin\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1476: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
C:\Users\admin\anaconda3\lib\site-packag

[CV 5/5] END LR__C=0.001, LR__l1_ratio=0.5005, LR__penalty=l1;, score=0.500 total time=   0.0s
[CV 1/5] END LR__C=0.001, LR__l1_ratio=0.5005, LR__penalty=l2;, score=0.813 total time=   0.0s
[CV 2/5] END LR__C=0.001, LR__l1_ratio=0.5005, LR__penalty=l2;, score=0.780 total time=   0.0s
[CV 3/5] END LR__C=0.001, LR__l1_ratio=0.5005, LR__penalty=l2;, score=0.858 total time=   0.0s
[CV 4/5] END LR__C=0.001, LR__l1_ratio=0.5005, LR__penalty=l2;, score=0.953 total time=   0.0s
[CV 5/5] END LR__C=0.001, LR__l1_ratio=0.5005, LR__penalty=l2;, score=0.982 total time=   0.0s
[CV 1/5] END LR__C=0.001, LR__l1_ratio=0.5005, LR__penalty=elasticnet;, score=0.500 total time=   0.0s
[CV 2/5] END LR__C=0.001, LR__l1_ratio=0.5005, LR__penalty=elasticnet;, score=0.500 total time=   0.0s
[CV 3/5] END LR__C=0.001, LR__l1_ratio=0.5005, LR__penalty=elasticnet;, score=0.500 total time=   0.0s
[CV 4/5] END LR__C=0.001, LR__l1_ratio=0.5005, LR__penalty=elasticnet;, score=0.500 total time=   0.0s
[CV 5/5] END LR__C

...

C:\Users\admin\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1476: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
C:\Users\admin\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\admin\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1476: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
C:\Users\admin\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\admin\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1476: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
C:\Users\admin\anaconda3\lib\site-packages\sklearn\linear_model\_sag

[CV 4/5] END LR__C=4.0, LR__l1_ratio=0.75025, LR__penalty=l1;, score=0.935 total time=   0.0s
[CV 5/5] END LR__C=4.0, LR__l1_ratio=0.75025, LR__penalty=l1;, score=0.893 total time=   0.0s
[CV 1/5] END LR__C=4.0, LR__l1_ratio=0.75025, LR__penalty=l2;, score=0.863 total time=   0.0s
[CV 2/5] END LR__C=4.0, LR__l1_ratio=0.75025, LR__penalty=l2;, score=0.802 total time=   0.0s
[CV 3/5] END LR__C=4.0, LR__l1_ratio=0.75025, LR__penalty=l2;, score=0.633 total time=   0.0s
[CV 4/5] END LR__C=4.0, LR__l1_ratio=0.75025, LR__penalty=l2;, score=0.935 total time=   0.0s
[CV 5/5] END LR__C=4.0, LR__l1_ratio=0.75025, LR__penalty=l2;, score=0.893 total time=   0.0s
[CV 1/5] END LR__C=4.0, LR__l1_ratio=0.75025, LR__penalty=elasticnet;, score=0.863 total time=   0.0s
[CV 2/5] END LR__C=4.0, LR__l1_ratio=0.75025, LR__penalty=elasticnet;, score=0.802 total time=   0.0s
[CV 3/5] END LR__C=4.0, LR__l1_ratio=0.75025, LR__penalty=elasticnet;, score=0.633 total time=   0.0s
[CV 4/5] END LR__C=4.0, LR__l1_ratio

C:\Users\admin\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\admin\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1476: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
C:\Users\admin\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\admin\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1476: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
C:\Users\admin\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\admin\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1476:

[CV 2/5] END LR__C=4.0, LR__l1_ratio=1.0, LR__penalty=l1;, score=0.802 total time=   0.0s
[CV 3/5] END LR__C=4.0, LR__l1_ratio=1.0, LR__penalty=l1;, score=0.639 total time=   0.0s
[CV 4/5] END LR__C=4.0, LR__l1_ratio=1.0, LR__penalty=l1;, score=0.935 total time=   0.0s
[CV 5/5] END LR__C=4.0, LR__l1_ratio=1.0, LR__penalty=l1;, score=0.893 total time=   0.0s
[CV 1/5] END LR__C=4.0, LR__l1_ratio=1.0, LR__penalty=l2;, score=0.863 total time=   0.0s
[CV 2/5] END LR__C=4.0, LR__l1_ratio=1.0, LR__penalty=l2;, score=0.802 total time=   0.0s
[CV 3/5] END LR__C=4.0, LR__l1_ratio=1.0, LR__penalty=l2;, score=0.633 total time=   0.0s
[CV 4/5] END LR__C=4.0, LR__l1_ratio=1.0, LR__penalty=l2;, score=0.935 total time=   0.0s
[CV 5/5] END LR__C=4.0, LR__l1_ratio=1.0, LR__penalty=l2;, score=0.893 total time=   0.0s
[CV 1/5] END LR__C=4.0, LR__l1_ratio=1.0, LR__penalty=elasticnet;, score=0.863 total time=   0.0s
[CV 2/5] END LR__C=4.0, LR__l1_ratio=1.0, LR__penalty=elasticnet;, score=0.802 total time=  

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=2022, shuffle=True),
             estimator=Pipeline(steps=[('STD', StandardScaler()),
                                       ('LR',
                                        LogisticRegression(random_state=100,
                                                           solver='saga'))]),
             param_grid={'LR__C': array([1.00000e-03, 1.00075e+00, 2.00050e+00, 3.00025e+00, 4.00000e+00]),
                         'LR__l1_ratio': array([0.001  , 0.25075, 0.5005 , 0.75025, 1.     ]),
                         'LR__penalty': ['l1', 'l2', 'elasticnet', None]},
             scoring='roc_auc', verbose=3)

In [47]:
print(gcv.best_params_)
print(gcv.best_score_)

{'LR__C': 0.001, 'LR__l1_ratio': 0.001, 'LR__penalty': 'elasticnet'}
0.881825866441251


In [48]:
y_pred = gcv.predict(x_test)